***
# Concatenating and merging data

More often than not, data sets come from various sources and need to be concatenated (the process of appending observations or variables) or merged as part of data pre-processing. Pandas offers several routines to accomplish such tasks which we study in this unit:

1. [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) allows us to combine multiple Series or DataFrames by appending observations (rows) or columns.
2. [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) allows us to match observations from one Series or DataFrame with observations from another Series or DataFrame and combine these into a _merged_ DataFrame.

You can also consult the official 
[user guide](https://pandas.pydata.org/docs/user_guide/merging.html) 
and the pandas 
[cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) 
for more information.

***
## Concatenation

Concatenation with [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) is used to combine multiple data sets along the row or column axes. This function can be called with both `Series` and `DataFrame` arguments, as we illustrate below.

### Concatenating Series

We begin with the simplest case of combining two `Series` to obtain a new `Series` which contains observations from both.

*Example: Concatenating two Series along the row axis*

In [1]:
import pandas as pd 

# Create first series of 3 observations
a = pd.Series(['A0', 'A1', 'A2'])
a

0    A0
1    A1
2    A2
dtype: object

In [2]:
# Data for second series (5 observations)
data_b = [f'B{i}' for i in range(5)]

# Create second series
b = pd.Series(data_b)
b

0    B0
1    B1
2    B2
3    B3
4    B4
dtype: object

To concatenate `a` and `b` along the first dimension, we call `pd.concat()` as follows:

In [3]:
# Call concat() with the default value for axis, which is axis=0
s = pd.concat((a, b))

# Alternatively, make explicit that we are concatenating along the row axis
# s = pd.concat((a, b), axis=0)
s

0    A0
1    A1
2    A2
0    B0
1    B1
2    B2
3    B3
4    B4
dtype: object

As you can see, `pd.concat()` also concatenates the index, which has the undesirable effect that the index values are no longer unique. As a rule, you never 
want to operate with non-unique indices in pandas as this can cause problems in some scenarios which are hard to diagnose.
We can rectify this with the [`reset_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) method that we encountered in previous units:

In [4]:
# Reset index to get rid of duplicates
s = s.reset_index(drop=True)
s

0    A0
1    A1
2    A2
3    B0
4    B1
5    B2
6    B3
7    B4
dtype: object

Alternatively, we can pass the `ignore_index=True` argument to `pd.concat()` and the return value will use the default index:

In [5]:
# Concatenate two series, use the default index for the return value
pd.concat((a, b), ignore_index=True)

0    A0
1    A1
2    A2
3    B0
4    B1
5    B2
6    B3
7    B4
dtype: object

*Example: Concatenating along the column axis*

It is also possible to concatenate `Series` along the column dimension by specifying `axis=1`. We would usually use this only for `Series` of equal length, as the result otherwise contains `NaN` values if the Series have different indices (e.g., because they differ in the number of observations).

In [6]:
s = pd.concat((a, b), axis=1)
s

,0,1
0,A0,B0
1,A1,B1
2,A2,B2
3,NaN,B3
4,NaN,B4


If the `Series` in question have no names, pandas assigns the values `0`, `1`, ... as column names. This can be avoided by explicitly passing the desired column names using the `keys` argument:

In [7]:
s = pd.concat((a, b), axis=1, keys=['Variable1', 'Variable2'])
s

,Variable1,Variable2
0,A0,B0
1,A1,B1
2,A2,B2
3,NaN,B3
4,NaN,B4


<div class="alert alert-info">
<h3> Your turn</h3>
<ol>
    <li>Create a new <TT>Series</TT> with observations <TT>['C0', 'C1']</TT>.</li>
    <li>Using the previously created <TT>Series</TT> <TT>a</TT> and <TT>b</TT>, concatenate all three objects along the row axis and create a new (unique) index.</li>
    <li>Repeat the previous step, but now concatenate along the column axis. Assign the column names <TT>'Column1'</TT>, <TT>'Column2'</TT>, and <TT>'Column3'</TT>.</li>
</ol>
</div>

***
### Concatenating DataFrames

Concatenating DataFrames works exactly the same way as for Series.

#### Concatenating along the column axis

*Example: Concatenating two DataFrames along the column axis*

In this example, we create two DataFrames with two and three columns, respectively.

In [8]:
import numpy as np

# Create 2 x 2 array of string data
data_a = np.array([f'A{i}' for i in range(4)]).reshape((2, 2))

df_a = pd.DataFrame(data_a)
df_a

,0,1
0,A0,A1
1,A2,A3


In [9]:
# Create 2 x 3 array of string data
data_b = np.array([f'B{i}' for i in range(6)]).reshape((2, 3))

df_b = pd.DataFrame(data_b)
df_b

,0,1,2
0,B0,B1,B2
1,B3,B4,B5


To create a new DataFrame which contains the columns from both `df_a` and `df_b`, we use `pd.concat(..., axis=1)`:

In [10]:
# Concatenate along the column axis
df = pd.concat((df_a, df_b), axis=1)
df

,0,1,0,1,2
0,A0,A1,B0,B1,B2
1,A2,A3,B3,B4,B5


As before, the resulting `DataFrame` can have non-unique column names which is undesirable. There is no `reset_index()` method for columns, but we can easily create unique column names as follows:

In [11]:
# Reset column index to 0, 1, 2,...
df.columns = np.arange(len(df.columns))
df

,0,1,2,3,4
0,A0,A1,B0,B1,B2
1,A2,A3,B3,B4,B5


Alternatively, we can pass the `ignore_index=True` argument to `pd.concat()` as we did earlier to have pandas create default column names.

In [12]:
# Concatenate columns, return DataFrame with default column names
pd.concat((df_a, df_b), axis=1, ignore_index=True)

,0,1,2,3,4
0,A0,A1,B0,B1,B2
1,A2,A3,B3,B4,B5


It is also possible to add an additional level of column names to the resulting `DataFrame` by specifying the `keys` argument:

In [13]:
# Concatenate along column axis, add additional column index level [A, B]
df = pd.concat((df_a, df_b), axis=1, keys=['A', 'B'])
df

A       B        
    0   1   0   1   2
0  A0  A1  B0  B1  B2
1  A2  A3  B3  B4  B5

The new `DataFrame` then has a so-called hierarchical column index.

*Example: Concatenating a DataFrame and a Series*

One can also concatenate DataFrames and Series object along the column axis. In that case, the `Series` is automatically converted to a `DataFrame` using the default column name.

In [14]:
s = pd.Series(['C0', 'C1'])
s

0    C0
1    C1
dtype: object

In [15]:
# Concatenate DataFrame and Series
pd.concat((df_a, s), axis=1, ignore_index=True)

,0,1,2
0,A0,A1,C0
1,A2,A3,C1


#### Concatenating along the row axis

We usually concatenate DataFrames along the row axis if we have observations on the same variables scattered across multiple data sets. Appending DataFrames with different columns will usually create `NaN` values and hence is often not useful.

*Example: Concatenating rows with identical columns*

In [16]:
#  Concatenate 2x2 DataFrame and 3x2 DataFrame (note the transpose!)
df = pd.concat((df_a, df_b.T), axis=0, ignore_index=True)
df

,0,1
0,A0,A1
1,A2,A3
2,B0,B3
3,B1,B4
4,B2,B5


*Example: Concatenating rows with different columns*

The DataFrames `df_a` and `df_b` have a different number of columns, so the resulting `DataFrame` will contain `NaN` for all observations of column `2` that were originally in `df_a`:

In [17]:
# Concatenate DataFrame rows with different numbers of columns
df = pd.concat((df_a, df_b), axis=0, ignore_index=True)
df

,0,1,2
0,A0,A1,NaN
1,A2,A3,NaN
2,B0,B1,B2
3,B3,B4,B5


<div class="alert alert-info">
<h3>Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>FRED_monthly_1950.csv</TT> and <TT>FRED_monthly_1960.csv</TT> into two different DataFrames.
        The files contain monthly macroeconomic time series for the 1950s and 1960s, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Concatenate these DataFrames along the row dimension to get a total of 240 observations.</li>
    <li>Set the column <TT>DATE</TT> as index for the newly created DataFrame.</li>
</ol>
</div>


***
## Merging and joining data sets

### Types of merges

While concatenation simply appends blocks of rows or columns from multiple data sets, merging allows for more fine-grained control over how data should be combined. The most common scenarios in empirical work are:

1.  *one-to-one*: The observations in data sets `A` and `B` have a unique identifier (_"key"_), and each observation in `A` is matched with at most one observation in `B`. For example, we could have data on individuals from multiple sources, and each of these data sets identifies individuals by their social security number. Each observation in one data set corresponds to exactly one observation in the other data set.
2.  *many-to-one*: Data set `A` contains unique identifiers but these can correspond to multiple observations in data set `B`. For example, we could have data at the ZIP-code (neighborhood) level in data set `A` and data on individuals in data set `B`. ZIP-codes are a unique identifier in `A`, but many individuals can live in the same neighboorhood, so each observation in `A` can reasonably be matched with many different observations in `B`.
3.  *many-to-many*: Identifying keys are not unique in either data set, and the resulting data set is a Cartesian product of all possible key combinations from both data sets. This situation should usually be *avoided* as it tends to have surprising results and can potentially consume large amounts of memory.

### Implementation in pandas

Merging in pandas can be performed in two different ways:

1.  [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) is a function that takes as argument the *two* DataFrames to be merged:
    
    ```python
    result = pd.merge(df_A, df_B)
    ```
2.  [`df.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) is a method of a specific `DataFrame` object, and takes as an 
    argument the other `DataFrame` to be merged:
    
    ```python
    result = df_A.merge(df_B)
    ```
    
Both ways are equivalent and can be used interchangeably.

### Controlling the resulting data set

Irrespective of whether we perform a _one-to-one_ or a _many-to-one_ merge, we frequently face the situation that some observations are present in one data set but not the other. We therefore need to control which subset of the data we want to retain in the final data set. This is accomplished using the `how` argument passed to `merge()`. There are several possible merge methods
which were originally introduced in SQL, a data processing language for relational databases
(see also the official [user guide](https://pandas.pydata.org/docs/user_guide/merging.html#merge-types)):

1.  `how='inner'` performs a so-called _inner join_: the merged data contains only the _intersection_ of keys that are present in _both_ data sets.
2.  `how='outer'` performs an _outer join_: the merged data contains the _union_ of keys present in either of the data sets. Rows which are not present in both data sets will contain missing values.
3.  `how='left'` performs a _left join_: all identifiers from the _left_ data set are present in the merge result, but 
    rows that are only present in the _right_ data set are dropped.
4.  `how='right'` performs a _right join_: all identifiers from the _right_ data set are present in the merge result, but 
    rows that are only present in the _left_ data set are dropped.

The following figure illustrates these concepts graphically using Venn diagrams. Each circle represents the keys present in the left (`df1`) or right (`df2`) DataFrames. The merge method controls which subset of keys is retained in the merge result.

![Join types](join-methods.png)

***
### Merging with `pd.merge()`

#### One-to-one merges

We first create two data sets `A` and `B` used to demonstrate various merge methods. We use the column `key` as the identifier on which to perform merges.

In [18]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})
df_a

,key,value_a
0,0,A0
1,1,A1


In [19]:
# Create second DataFrame with 2 rows
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})
df_b

,key,value_b
0,1,B1
1,2,B2


When merging two DataFrames, in most cases we need to specify the columns (or index levels) on which the merge should be performed. We do this using the argument `on` when calling `pd.merge()`.

In [20]:
# Merge A and B on the identifier 'key' using an inner join
pd.merge(df_a, df_b, on='key', how='inner')

,key,value_a,value_b
0,1,A1,B1


Note that in this case we could leave the `on` argument unspecified, as then `pd.merge()` by default merges on the intersection of columns present in both DataFrames (which in this case is just the column `key`). However, for clarity it is advisable to always specify `on` explicitly.

Moreover, `pd.merge()` performs an inner join by default, so we could have called the function as follows to get the same result:

In [21]:
# Merge A and B on default key using default inner join
pd.merge(df_a, df_b)

,key,value_a,value_b
0,1,A1,B1


Since we are performing an inner join, the merged data set contains only a single column corresponding to the identifier `1`, the only one present on both DataFrames.

If we want to retain all observations, we achieve this using an outer join:

In [22]:
# Merge A and B using outer join (keep union of observations)
pd.merge(df_a, df_b, on='key', how='outer')

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


Since the keys `0` and `2` are not present in both DataFrames, the corresponding columns contain missing values.

We can also only retain the keys present in the left (i.e., the first argument) or the right (i.e., the second argument) DataFrame:

In [23]:
# Merge A and B on the identifier 'key', keep left identifiers
pd.merge(df_a, df_b, on='key', how='left')

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


In [24]:
# Merge A and B on the identifier 'key', keep right identifiers
pd.merge(df_a, df_b, on='key', how='right')

,key,value_a,value_b
0,1,A1,B1
1,2,NaN,B2


<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Merge the CPI and the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.merge.html"><TT>pd.merge()</TT></a> 
    without specifying the <TT>how=</TT> argument.</li>
    <li>Merge the CPI and the GDP time series with <TT>pd.merge()</TT>
    using a left join (<TT>how='left'</TT>).</li>
    <li>Merge the CPI and the GDP time series with <TT>pd.merge()</TT> using an inner join (<TT>how='inner'</TT>).</li>
</ol>
How do the results differ depending on the value of the <TT>how</TT> argument?
</div>

#### Many-to-one merges

To illustrate many-to-one merges, we create a DataFrame `A` which has non-unique values in the column `keys`. We want to merge this with another DataFrame `B` with unique values in the column `keys`. This operation is therefore a many-to-one merge.

In [25]:
# Create first DataFrame with 4 rows, non-unique key
df_a = pd.DataFrame({'key': [0, 1, 0, 1], 'value_a': ['A0', 'A1', 'A2', 'A3']})
df_a

,key,value_a
0,0,A0
1,1,A1
2,0,A2
3,1,A3


In [26]:
# Create second DataFrame with 2 rows, unique key
df_b = pd.DataFrame({'key': [0, 1], 'value_b': ['B0', 'B1']})
df_b

,key,value_b
0,0,B0
1,1,B1


In [27]:
# Merge A and B on the identifier 'key', keep left identifiers
pd.merge(df_a, df_b, on='key')

,key,value_a,value_b
0,0,A0,B0
1,1,A1,B1
2,0,A2,B0
3,1,A3,B1


As you can see, in the resulting DataFrame the values from `B` are matched with multiple rows of `A`.

***
### Merging with `DataFrame.merge()`

As mentioned above, there is an alternative but equivalent way to merge DataFrames using the method
[`df.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).
In this context, the _left_ `DataFrame` is the one on which `merge()` is being invoked, while the _right_ `DataFrame`
is the argument passed to `merge()`.

To illustrate, we recreate the DataFrames we used for the one-to-one merge examples above:

In [28]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})

# Create second DataFrame with 2 rows
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

We now use the `merge()` method invoked on DataFrame `A` to merge it with DataFrame `B`:

In [29]:
# Use DataFrame method to merge, keep only left identifiers
df_a.merge(df_b, on='key', how='left')

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


In [30]:
# Now df_a is the right DataFrame, set of final identifiers is the same as
# in the example above!
df_b.merge(df_a, on='key', how='right')

,key,value_b,value_a
0,0,NaN,A0
1,1,B1,A1


*Example: Merging with overlapping column names*

Sometimes both DataFrames contain the same column names. If these columns are not used as keys in the merge operation, pandas automatically renames these columns in the resulting `DataFrame` to avoid naming clashes.

To illustrate, we rename the value columns to `'value'` in both DataFrames and then perform the merge:

In [31]:
# Rename columns to common name 'value'
df_a = df_a.rename(columns={'value_a': 'value'})
df_b = df_b.rename(columns={'value_b': 'value'})

Note that once we have identical column names `['key', 'value']` in both DataFrames, we _must_ specify the `on` argument to `merge()` as otherwise pandas by default merges on the intersection on column names in both DataFrames, i.e., in this case it merges on `['key', 'value']`:

In [32]:
# Invoking merge() with default on argument has unintended consequences
df_a.merge(df_b)

,key,value


The merge result is empty because we are performing an _inner join_ (the default), and there are no overlapping rows that have the same values for both `key` and `value` columns. We therefore need to explicitly specify `on='key'` to get the desired result:

In [33]:
# Merge DataFrames with overlapping column 'value'
df_a.merge(df_b, on='key')

,key,value_x,value_y
0,1,A1,B1


As you can see, pandas automatically appends the suffixes `'_x'` and `'_y'` to the column from the left and right DataFrames, respectively. we can change this default behavior by explicitly specifying the suffixes to be appended:

In [34]:
df_a.merge(df_b, on='key', suffixes=('_left', '_right'))

,key,value_left,value_right
0,1,A1,B1


<div class="alert alert-info">
<h3> Your turn</h3>
Repeat the previous exercise, but use the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html"><TT>DataFrame.merge()</TT></a> 
method to perform the merge.
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP without specifying the <TT>how=</TT> argument.</li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP using a <i>left</i> join.</li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP using an <i>inner</i> join.</li>
</ol>
How do the results differ depending on the value of the <TT>how</TT> argument?
</div>

***
### Merging with `join()`

The `DataFrame` method 
[`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) 
is a convenience wrapper around 
[`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
with the following subtle differences:

1.  `join()` can be called _only_ directly on the `DataFrame` object, i.e., `df.join()`, while for merge we have both the `pd.merge()`
    and the `df.merge()` variants.
2.  `join()` always operates on the _index_ of the other `DataFrame`, whereas `merge()` is more flexible and can operate on either the index or on columns.
3.  `join()` by default performs a `left` join, whereas `merge()` performs an `inner` join.

As a rule of thumb, you should use `join()` if you want to join DataFrames which have a similar index.

*Example: joining DataFrames*

We first create two DataFrames to be joined. This time, we explicitly set an index for each of them which will be used to perform the `join()`.

In [35]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame(['A0', 'A1'], columns=['value_a'], index=[0, 1])
df_a

,value_a
0,A0
1,A1


In [36]:
# Create second DataFrame with 2 rows
df_b = pd.DataFrame(['B1', 'B2'], columns=['value_b'], index=[1, 2])
df_b

,value_b
1,B1
2,B2


In [37]:
# Perform left join (the default option)
df_a.join(df_b)

,value_a,value_b
0,A0,NaN
1,A1,B1


In [38]:
# Join with explicit inner join
df_a.join(df_b, how='inner')

,value_a,value_b
1,A1,B1


In [39]:
# Perform an outer join
df_a.join(df_b, how='outer')

,value_a,value_b
0,A0,NaN
1,A1,B1
2,NaN,B2


<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Set the <TT>DATE</TT> column as the index for each of the two DataFrames.</li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html"><TT>join()</TT></a>. 
    Do this with both a left and an inner join.</li>
</ol>
</div>

***
# Dealing with missing values

We already encountered missing values in earlier lectures. These are particularly likely to arise when merging or concatenating data if individual DataFrames lack some observations. 

To illustrate, recall the example from above:

In [40]:
# Create two DataFrames with partially overlapping keys
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

In [41]:
# Perform outer merge, keep union of keys
pd.merge(df_a, df_b, on='key', how='outer')

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


Since they keys in DataFrames `df_a` and `df_b` were only partially overlapping, the resulting DataFrame has missing values by construction. In what follows, we explore strategies on how to handle these missing data.

## Dropping missing values

One strategy is to drop missing values outright, even though we might lose information that could be useful to perform data analysis if only some but not all columns contain missing values, as is the case above.

Missing values can be dropped by either

1.  Using [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html), potentially restricted to a subset of columns.
2.  Selecting a subset of observations to keep with a boolean operation such as 
    [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html) or
    [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html).
3. Avoiding the missing values in the first place, e.g., by using `merge(..., how='inner')`.

*Example: Dropping missing values*

Consider the merged `DataFrame` from above. We can drop rows with missing values with [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html), which by default drops all rows with _any_ missing values. Alternatively, we can specify only a subset of columns to be checked for missing values.

In [42]:
# Merge with outer join, thus creating missing values
df = pd.merge(df_a, df_b, on='key', how='outer')

In [43]:
# Drop any row which contains at least one missing value
df.dropna()

,key,value_a,value_b
1,1,A1,B1


In [44]:
# Drop rows which contain missing values in column 'value_a', ignore missing
# values in 'value_b'
df.dropna(subset='value_a')

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


<div class="alert alert-info">
<h3> Your turn</h3>
Instead of using <TT>dropna()</TT>, drop the missing observations using either <TT>notna()</TT> or <TT>isna()</TT>.
<ol>
    <li>Drop all rows with any missing observations.</li>
    <li>Drop all rows with missing observations in column <TT>value_a</TT>.</li>
</ol>
</div>

*Example: Avoiding missing values in the first place*

Of course the missing values in the example above arose only because we specified `how='outer'`. Merging with `how='inner'` drops keys which are not present in both DataFrames right away, avoiding the issue of missing values (unless these are already present in the original DataFrames):

In [45]:
# Merge using inner join, drop keys not present in both DataFrames
pd.merge(df_a, df_b, on='key', how='inner')

,key,value_a,value_b
0,1,A1,B1


## Filling missing values

Instead of dropping data, we can impute missing values in various ways:

1.  [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) can be used to replace missing data with user-specified values.
2.  [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) and 
    [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) can be used to fill missing values
    forward or backward from adjacent non-missing observations.
3.  [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html) supports various interpolation methods such as linear interpolation based on non-missing values.

*Example: Replacing missing values with `fillna()`*

Consider the merged `DataFrame` we have created above:

In [46]:
df = pd.merge(df_a, df_b, on='key', how='outer')
df

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


We can use [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) to replace missing values with some constant.

In [47]:
# Replace ALL missing values with 'Some value'
df.fillna('Some value')

,key,value_a,value_b
0,0,A0,Some value
1,1,A1,B1
2,2,Some value,B2


This might not be what you want as the provided non-missing value is imposed on _all_ columns. It is therefore possible to specify a different value for each column using a dictionary as an argument.

In [48]:
# Use different replacement values for columns 'value_a' and 'value_b'
df.fillna({'value_a': 'Missing A', 'value_b': 'Missing B'})

,key,value_a,value_b
0,0,A0,Missing B
1,1,A1,B1
2,2,Missing A,B2


*Example: forward- or backward-filling missing values*

Another common imputation method is to use the previous (_"forward"_) or next (_"backward"_) non-missing value as replacement for missing data.

Continuing with the `DataFrame` from the previous example, we can apply these methods as follows:

In [49]:
# Forward-fill missing values from previous observation
df.ffill()

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,A1,B2


This inserts the value `'A1'` in the 3rd row of column `value_a`, but does not do anything about the missing value in column `value_b` since there is no preceding non-missing value.

Conversely, `bfill()` does the opposite and backfills the missing value in column `value_b`:

In [50]:
df.bfill()

,key,value_a,value_b
0,0,A0,B1
1,1,A1,B1
2,2,NaN,B2


*Example: linear interpolation*

Consider the following `Series` with numerical data (interpolation only makes sense for numerical data, not strings):

In [51]:
s = pd.Series([1.0, 2.0, 3.0, np.nan, 5.0])
s

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

We can interpolate the missing data using 
[`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html),
for example by using linear interpolation (check the documentation for many other interpolation methods).

In [52]:
# Interpolate missing values using linear interpolation
s.interpolate(method='linear')

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using a left join. This creates missing values in the <TT>GDP</TT>
    column.</li>
    <li>Impute the missing GDP values using <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html"><TT>interpolate()</TT></a> 
    and replace the missing values in column <TT>GDP</TT>.</li>
</ol>
</div>

***
# List of functions used in this lecture

The following list contains the central functions covered in this lecture. Each function name is linked to the official API documentation which you can consult for more details regarding function arguments and return values. The [[go to section]]() links to the relevant section in this notebook.

## Concatenating (appending) data

-   [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) —
    append Series or DataFrames along the row or column dimension [[go to section]](#concatenation)
-   [`reset_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) —
    reset (potentially non-unique) index after concatenation [[go to section]](#concatenation)

## Merging data

-   [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) —
    merge two Series or DataFrames along the column dimension based some keys [[go to section]](#merging-with-pdmerge)
-   [`DataFrame.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) —
    merge a given (left) DataFrame with another (right) Series or DataFrame based on some keys [[go to section]](#merging-with-dataframemerge)
-   [`DataFrame.join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) —
    merge a given (left) DataFrame with another (right) Series or DataFrame based on the index [[go to section]](#merging-with-join)

## Dealing with missing values

-   [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html) —
    check whether (one or more) columns have non-missing values [[go to section]](#dropping-missing-values)
-   [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html) —
    check whether (one or more) columns have missing values [[go to section]](#dropping-missing-values)
-   [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) —
    drop rows with missing observations [[go to section]](#dropping-missing-values)
-   [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) —
    replace missing values with a given value [[go to section]](#filling-missing-values)
-   [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) —
    fill missing values by propagating the last valid observation [[go to section]](#filling-missing-values)
-   [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) —
    fill missing values by using the next valid observation to fill the gap [[go to section]](#filling-missing-values)
-   [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html) —
    interpolate missing values from adjacent non-missing ones [[go to section]](#filling-missing-values)